https://blog.netcetera.com/object-detection-and-tracking-in-2020-f10fb6ff9af3

The computer vision researching community has shifted focus in other very interesting and challenging topics, such as ***adversarial image generation, neural style transfer, visual storytelling,*** and of course, object detection, segmentation and tracking.

> 저런 주제에 대해서도 내용은 알고는 있어야할듯!

The classic feature detection, combined with a machine learning algorithm like KNN or SVM for classification, or with a description matcher like FLANN for object detection.
The most notable feature detection algorithms are arguably SIFT and SURF as feature descriptors, and FAST for corner detection. The feature descriptors use a series of mathematical approximations to learn a representation of the image that is scale-invariant. Some of these old school methods could sometimes get the job done, but there is a lot more we can do.

> 어차피 surf, sift, fast 모두 비선형의 커널 같은 느낌이니 단순히 이를 뉴럴 넷으로 만들어서 모델로서 instace가 되도록 하면 어떨까??



As for object tracking, it seems like the traditional methods stood the test of time better than the object detection ones. Ideas like Kalman filtering, sparse and dense optical flow are still in widespread use. Kalman filtering entered hall of fame when it was used in the Apollo PGNCS to produce an optimal position estimate for the spacecraft, based on past position measurements and new data. Its influence can be still seen today in many algorithms, such as the Simple Online and Realtime Tracking (SORT), which uses a combination of the Hungarian algorithm and Kalman filter to achieve decent object tracking.

### The Novel Advancements of Object Tracking

#### ROLO
For starters, we can check out Recurrent [YOLO](https://arxiv.org/abs/1607.05781) (ROLO), a single object tracking method that combines object detection and recurrent neural networks. ROLO is a combination of YOLO and LSTM. The object detection module uses YOLO to collect visual features, along with location inference priors. At each time-step (frame), the LSTM receives an input feature vector of length 4096, and returns the location of the tracked object.

#### SiamMask
When it comes to single object tracking, [SiamMask](https://arxiv.org/abs/1812.05050) is an excellent choice. It is based on the charming siamese neural network, which rose in popularity with [Google’s Facenet](https://blog.netcetera.com/face-recognition-using-one-shot-learning-a7cf2b91e96c). Besides producing rotated bounding boxes at 55 frames per second, it also provides class-agnostic object segmentation masks. In order to achieve this, SiamMask needs to be initialized with a single bounding box so it can track the desired object. However, this also means that multiple object tracking (MOT) is not viable with SiamMask, and ***modifying the model to support that will leave us with a significantly slower object detector.***

> modify the model to support???

There are a couple of other notable object trackers that utilize siamese neural networks, such as [DaSiamRPN](https://arxiv.org/abs/1808.06048), which won the [VOT-18 challenge](https://www.votchallenge.net/vot2018/) [(PyTorch 0.3.1 code)](https://github.com/foolwood/DaSiamRPN) and [SiamDW](https://arxiv.org/abs/1901.01660) [(PyTorch 0.3.1 code)](https://github.com/shallowtoil/SiamDW-pytorch).

#### Deep SORT

[Deep SORT](https://arxiv.org/abs/1703.07402) is improving SORT by replacing the associating metric with a novel cosine metric learning, a method for learning a feature space where the cosine similarity is effectively optimized through reparametrization of the softmax regime.
The track handling and Kalman filtering framework is almost identical to the original SORT, except the bounding boxes are computed using a pre-trained convolutional neural network, trained on a large-scale person re-identification dataset. This method is a great starting point for multiple object detection, as it is simple to implement, offers solid accuracy, but above all, runs in real-time.

> Kalman filter가 NN으로 된게 있으니까 이걸 적용???

The track handling and Kalman filtering framework is almost identical to the original SORT, except the bounding boxes are computed using a pre-trained convolutional neural network, trained on a large-scale person re-identification dataset. This method is a great starting point for multiple object detection, as it is simple to implement, offers solid accuracy, but above all, runs in real-time.

#### TrackR-CNN

[TrackR-CNN](https://www.vision.rwth-aachen.de/media/papers/mots-multi-object-tracking-and-segmentation/MOTS.pdf) was introduced just as a baseline for the [Multi Object Tracking and Segmentation (MOTS) challenge](https://www.vision.rwth-aachen.de/page/mots), but it turns out that it is actually effective. First off, the object detection module utilizes Mask R-CNN on top of a ResNet-101 backbone. The tracker is created by integrating 3D convolutions that are applied to the backbone features, incorporating temporal context of the video. As an alternative, convolutional LSTM is considered as well, but the latter method does not yield any gains compared with the baseline.
TrackR-CNN also extends Mask R-CNN by an association head, to be able to associate detections over time. This is a fully connected layer that receives region proposals and outputs an association vector for each proposal. The association head draws inspiration from siamese networks and the embedding vectors used in person re-identification. It is trained using a video sequence adaptation of batch hard triplet loss, which is a more efficient method than the original triplet loss. To produce the final result, the system must decide which detections should be reported. The matching between the previous frame detections and current proposals is done using the Hungarian algorithm, while only allowing pairs of detections with association vectors smaller than some threshold.

#### Tracktor++
The Multiple Object Tracking Benchmark makes it easier to find the most recent breakthroughs in MOT, thanks to its public leaderboard. The CVPR 2019 Tracking Challenge motivated progress in both accuracy and speed of the trackers. Tracktor++ dominated the leaderboard with a very simple, yet effective approach. This model predicts the position of an object in the next frame by calculating the bounding box regression, without needing to train or optimize on tracking data whatsoever. The object detector for Tracktor++ is the usual Faster R-CNN with 101-layer ResNet and FPN, trained on the MOT17Det pedestrian detection dataset.

